<span style="font-size:1.15em;font-weight:bold">
Your whole solution to the exercise should be implemented in this
file.

Be sure to read all comments in this file</span>

Your task is to first implement the functions `parse_json`,
`parse_timestamp`, `get_uuid`, `parse_bson`, `parse_xml`, and
`parse_volts` based on the information defined in the function
docstring.

Finally implement the main `solution` function.

Only the libraries mentioned in the Pipfile (in addition to the python
standard libraries) will be available in our grading setup



In [1]:
%pip install bson

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ETCE.GroupA.Ex04.weather_data as wd  # Do not remove this import
from pprint import pprint

In [3]:
import json
import bson
import xml.etree.ElementTree as ET
import datetime
import pandas as pd
import uuid

Implement this function to parse the json weather data here,
and return a dictionary as follows:


    {'temp': ...,
     'clouds', ...,
     'wind_speed': ...,
     'wind_deg': ...,
     'wind_gust': ...,
     'rain': ...
    }


In [4]:
def parse_json(string_returned) -> dict:
    # return my_dict
    pass

Implement this function, to parse the string timestamp 'yyyy-mm-dd hh:mm:ss.ms CEST' 
to a UNIX timestamp in UTC

In [5]:
def parse_timestamp(string_timestamp) -> int:
    # return int(...)
    pass

Implement this function to get a UUID using the uuid module

In [6]:
def get_uuid() -> str:
    # return str(...)
    pass

Implement this function to parse the bson weather data here,
and return a dictionary as follows:


    {'temp': ...,
     'clouds', ...,
     'wind_speed': ...,
     'wind_deg': ...,
     'wind_gust': ...,
     'rain': ...
    }

In [7]:
def parse_bson(string_returned) -> dict:
    # return my_dict
    pass

Implement this function to parse the xml weather data here,
and return a dictionary as follows:


    {'temp': ...,
     'clouds', ...,
     'wind_speed': ...,
     'wind_deg': ...,
     'wind_gust': ...,
     'rain': ...
    }

In [8]:
def parse_xml(string_returned) -> dict:
    # return my_dict
    pass

Implement this function to parse the xml voltage data from the
TMP36 sensor here, convert the voltage to temperature in °C and
return a dictionary as follows:    

{'temp': ....}

Hint: Search for a datasheet for a TMP36 sensor that shows the
relationship between a voltage reading from the sensor and the
temperature being sensed.

In [9]:
def parse_volts(string_returned)-> dict:
    # return my
    pass

This function should return the solution to E03.

This function should return a dictionary as follows:

    {0 : {"uuid" : "some-uuid",   Should be unique for every entry
          "timestamp": 00000000,  Some valid UNIX timestamp
           "temp":5.33,
           "clouds":100.0,
           "wind_speed":2.48,
           "wind_deg":31.0,
           "wind_gust":4.1,
           "rain":1.54
           } ,
     1 : {...},
     .
     .
     .
     n : {...}
     }

    where n=total no of timestamps

    Remember that you are combining data from various sources into one
    dict for each timestamp.

    When new data from multiple sources is available for the same
    field in the data structure, compute the average. For example, all
    sources give you temperature data, so you must return the average
    temperature for each timestamp.

    Also remember that you need to convert the voltage values for the
    TMP36 sensor into temperature in °C before averaging with the
    other temperature values.

    Side note: Some cloudiness (%) data might go above 100 or below
    0. You can simply ignore this mistake and proceed assuming a value
    like 100.05 is correct.

In [10]:
def solution()-> dict:
    # You can iteratively populate the solution_dict
    solution_dict = {}

    while(True):
        try:
            ret, ts = wd.get_next()
            # This function simulates an API query to get the weather
            # data.

            # `ret` is a tuple containing weather data from 5 sources,
            # (Source 1, Source 2, Source 3, Source 4, Source 5) at
            # timestamp `ts`

            # ts is the string timestamp when the data was recorded.

            # The structure of `ret` is self-documented, and contains
            # all the info you need to parse the data.

            # You can read the data returned by the API as follows
            # pprint(ret[0]["description"])
            # print(type(ret[0]["data"]))

        except Exception as E:
            print(E)
            break

    return solution_dict

In [11]:
from driver import evaluate
evaluate(solution, parse_json, parse_bson, parse_xml, parse_timestamp, get_uuid, parse_volts)

No more weather data
E04 task failed to run:


FileNotFoundError: [Errno 2] No such file or directory: 'ga.sol.json'